In [151]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
df1 = pd.read_csv(r"https://raw.githubusercontent.com/Perovsky56/BankLoanModel/refs/heads/main/datasets/Loan.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
print(df1.columns)

Index(['ApplicationDate', 'Age', 'AnnualIncome', 'CreditScore', 'EmploymentStatus',
       'EducationLevel', 'Experience', 'LoanAmount', 'LoanDuration', 'MaritalStatus',
       'NumberOfDependents', 'HomeOwnershipStatus', 'MonthlyDebtPayments',
       'CreditCardUtilizationRate', 'NumberOfOpenCreditLines', 'NumberOfCreditInquiries',
       'DebtToIncomeRatio', 'BankruptcyHistory', 'LoanPurpose', 'PreviousLoanDefaults',
       'PaymentHistory', 'LengthOfCreditHistory', 'SavingsAccountBalance',
       'CheckingAccountBalance', 'TotalAssets', 'TotalLiabilities', 'MonthlyIncome',
       'UtilityBillsPaymentHistory', 'JobTenure', 'NetWorth', 'BaseInterestRate', 'InterestRate',
       'MonthlyLoanPayment', 'TotalDebtToIncomeRatio', 'LoanApproved', 'RiskScore'],
      dtype='object')


In [ ]:
print(df1.head(3))

  ApplicationDate  Age  AnnualIncome  CreditScore EmploymentStatus EducationLevel  Experience  \
0      2018-01-01   45         39948          617         Employed         Master          22   
1      2018-01-02   38         39709          628         Employed      Associate          15   
2      2018-01-03   47         40724          570         Employed       Bachelor          26   

   LoanAmount  LoanDuration MaritalStatus  NumberOfDependents HomeOwnershipStatus  \
0       13152            48       Married                   2                 Own   
1       26045            48        Single                   1            Mortgage   
2       17627            36       Married                   2                Rent   

   MonthlyDebtPayments  CreditCardUtilizationRate  NumberOfOpenCreditLines  \
0                  183                   0.354418                        1   
1                  496                   0.087827                        5   
2                  902              

In [ ]:
df1 = df1.drop(['ApplicationDate', 'CreditScore', 'HomeOwnershipStatus', 'CreditCardUtilizationRate', 'NumberOfOpenCreditLines', 'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'LoanPurpose', 'PreviousLoanDefaults', 'PaymentHistory', 'LengthOfCreditHistory', 'UtilityBillsPaymentHistory', 'BaseInterestRate', 'InterestRate', 'TotalDebtToIncomeRatio', 'LoanApproved', 'RiskScore'], axis=1)

In [ ]:
df1.head(3)

,Age,AnnualIncome,EmploymentStatus,EducationLevel,Experience,LoanAmount,LoanDuration,MaritalStatus,NumberOfDependents,MonthlyDebtPayments,BankruptcyHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,JobTenure,NetWorth,MonthlyLoanPayment
0,45,39948,Employed,Master,22,13152,48,Married,2,183,0,7632,1202,146111,19183,3329.000000,11,126928,419.805992
1,38,39709,Employed,Associate,15,26045,48,Single,1,496,0,4627,3460,53204,9595,3309.083333,3,43609,794.054238
2,47,40724,Employed,Bachelor,26,17627,36,Married,2,902,0,886,895,25176,128874,3393.666667,6,5205,666.406688


In [ ]:
columns_to_move = ['LoanAmount', 'LoanDuration']

cols = [col for col in df1.columns if col not in columns_to_move] + columns_to_move
df1 = df1[cols]

In [ ]:
df1.sample(15)

,Age,AnnualIncome,EmploymentStatus,EducationLevel,Experience,MaritalStatus,NumberOfDependents,MonthlyDebtPayments,BankruptcyHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,JobTenure,NetWorth,MonthlyLoanPayment,LoanAmount,LoanDuration
17438,34,15000,Employed,High School,11,Single,2,303,0,5224,1081,246899,120429,1250.000000,3,126470,331.696290,10127,48
19652,45,18625,Employed,Master,23,Divorced,4,562,0,7005,3400,34354,170386,1552.083333,3,10668,171.132861,7640,96
14444,35,56121,Employed,Bachelor,12,Single,0,286,0,4150,6119,92299,22512,4676.750000,4,69787,1095.300185,33695,48
9782,26,53486,Employed,Bachelor,5,Married,0,521,0,9876,995,198772,92154,4457.166667,5,106618,616.710704,21065,60
3689,38,17473,Employed,Associate,13,Married,2,179,0,2210,779,31650,17593,1456.083333,2,14057,983.667047,39299,108
5735,36,45457,Employed,Bachelor,11,Widowed,3,900,1,2843,45,94948,44987,3788.083333,4,49961,3162.456966,73730,36
8054,33,140647,Employed,Bachelor,9,Married,1,535,0,3349,381,23416,33150,11720.583333,6,5247,1282.613648,33663,36
9788,21,15000,Employed,High School,2,Married,1,890,0,9409,473,104058,105223,1250.000000,5,5293,943.504069,27847,48
18135,22,21971,Employed,High School,0,Married,1,387,0,7743,2575,67257,12783,1830.916667,5,54474,2903.534386,31146,12
18787,59,58487,Employed,High School,38,Divorced,2,610,0,1315,984,24292,28365,4873.916667,5,9733,892.262015,31215,84


In [ ]:
df1.shape

(20000, 19)

In [ ]:
df1.dtypes

,0
Age,int64
AnnualIncome,int64
EmploymentStatus,object
EducationLevel,object
Experience,int64
MaritalStatus,object
NumberOfDependents,int64
MonthlyDebtPayments,int64
BankruptcyHistory,int64
SavingsAccountBalance,int64


In [ ]:
df1.describe()

,Age,AnnualIncome,Experience,NumberOfDependents,MonthlyDebtPayments,BankruptcyHistory,SavingsAccountBalance,CheckingAccountBalance,TotalAssets,TotalLiabilities,MonthlyIncome,JobTenure,NetWorth,MonthlyLoanPayment,LoanAmount,LoanDuration
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,2.000000e+04,2.000000e+04,20000.000000,20000.000000,2.000000e+04,20000.000000,20000.000000,20000.000000
mean,39.752600,59161.473550,17.522750,1.517300,454.292700,0.052400,4946.052150,1782.555100,9.696440e+04,3.625241e+04,4891.715521,5.002650,7.229432e+04,911.607052,24882.867800,54.057000
std,11.622713,40350.845168,11.316836,1.386325,240.507609,0.222838,6604.887477,2245.378812,1.207999e+05,4.725151e+04,3296.771598,2.236804,1.179200e+05,674.583473,13427.421217,24.664857
min,18.000000,15000.000000,0.000000,0.000000,50.000000,0.000000,73.000000,24.000000,2.098000e+03,3.720000e+02,1250.000000,0.000000,1.000000e+03,97.030193,3674.000000,12.000000
25%,32.000000,31679.000000,9.000000,0.000000,286.000000,0.000000,1541.750000,551.000000,3.118025e+04,1.119675e+04,2629.583333,3.000000,8.734750e+03,493.763700,15575.000000,36.000000
50%,40.000000,48566.000000,17.000000,1.000000,402.000000,0.000000,2986.000000,1116.000000,6.069900e+04,2.220300e+04,4034.750000,5.000000,3.285550e+04,728.511452,21914.500000,48.000000
75%,48.000000,74391.000000,25.000000,2.000000,564.000000,0.000000,5873.250000,2126.000000,1.174052e+05,4.314650e+04,6163.000000,6.000000,8.882550e+04,1112.770759,30835.000000,72.000000
max,80.000000,485341.000000,61.000000,5.000000,2919.000000,1.000000,200089.000000,52572.000000,2.619627e+06,1.417302e+06,25000.000000,16.000000,2.603208e+06,10892.629520,184732.000000,120.000000


In [ ]:
nulls_summary = pd.DataFrame(df1.isnull().any(), columns=['Nulls'])
nulls_summary['Num_of_nulls [qty]'] = pd.DataFrame(df1.isnull().sum())
nulls_summary['Num_of_nulls [%]'] = round((df1.isnull().mean()*100),2)
print(nulls_summary)

                        Nulls  Num_of_nulls [qty]  Num_of_nulls [%]
Age                     False                   0               0.0
AnnualIncome            False                   0               0.0
EmploymentStatus        False                   0               0.0
EducationLevel          False                   0               0.0
Experience              False                   0               0.0
MaritalStatus           False                   0               0.0
NumberOfDependents      False                   0               0.0
MonthlyDebtPayments     False                   0               0.0
BankruptcyHistory       False                   0               0.0
SavingsAccountBalance   False                   0               0.0
CheckingAccountBalance  False                   0               0.0
TotalAssets             False                   0               0.0
TotalLiabilities        False                   0               0.0
MonthlyIncome           False                   

In [ ]:
mapping1 = {'Employed': 0, 'Self-Employed': 1, 'Unemployed': 2}
mapping2 = {'High School': 0, 'Associate': 1, 'Bachelor': 2, 'Master': 3, 'Doctorate': 4}
mapping3 = {'Single': 0, 'Married': 1, 'Divorced': 2, 'Widowed': 3}

df1['EmploymentStatus'] = df1['EmploymentStatus'].replace(mapping1)
df1['EducationLevel'] = df1['EducationLevel'].replace(mapping2)
df1['MaritalStatus'] = df1['MaritalStatus'].replace(mapping3)
print(df1[['EmploymentStatus']])
df1.dtypes

       EmploymentStatus
0                     0
1                     0
2                     0
3                     0
4                     0
...                 ...
19995                 0
19996                 0
19997                 0
19998                 0
19999                 0

[20000 rows x 1 columns]


<ipython-input-38-201dfa7537d8>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['EducationLevel'] = df1['EducationLevel'].replace(mapping2)
<ipython-input-38-201dfa7537d8>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['MaritalStatus'] = df1['MaritalStatus'].replace(mapping3)


,0
Age,int64
AnnualIncome,int64
EmploymentStatus,int64
EducationLevel,int64
Experience,int64
MaritalStatus,int64
NumberOfDependents,int64
MonthlyDebtPayments,int64
BankruptcyHistory,int64
SavingsAccountBalance,int64


In [137]:
excluded_columns = ['LoanDuration', 'LoanAmount']
selected_columns = [col for col in df1.columns if col not in excluded_columns]

X = df1[selected_columns]
y = df1[['LoanAmount', 'LoanDuration']]

# Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [150]:
base_model = RandomForestRegressor(n_estimators=100, random_state=42)
model = MultiOutputRegressor(base_model)

# Trenowanie modelu
model.fit(X_train, y_train)

MultiOutputRegressor(estimator=RandomForestRegressor(n_estimators=300,
                                                     random_state=42))

In [160]:
model2 = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, random_state=42))
model2.fit(X_train, y_train)

MultiOutputRegressor(estimator=GradientBoostingRegressor(random_state=42))

In [162]:
y_pred = model.predict(X_test)
y_pred2 = model2.predict(X_test)

In [156]:
mse_loan_amount = mean_squared_error(y_test["LoanAmount"], y_pred[:, 0])
mse_loan_duration = mean_squared_error(y_test["LoanDuration"], y_pred[:, 1])
mae_loan_amount = mean_absolute_error(y_test["LoanAmount"], y_pred[:, 0])
mae_loan_duration = mean_absolute_error(y_test["LoanDuration"], y_pred[:, 1])
r2_loan_amount = r2_score(y_test["LoanAmount"], y_pred[:, 0])
r2_loan_duration = r2_score(y_test["LoanDuration"], y_pred[:, 1])

print(f'MSE Loan Amount: {mse_loan_amount}')
print(f'MSE Loan Duration: {mse_loan_duration}')
print(f'MAE Loan Amount: {mae_loan_amount}')
print(f'MAE Loan Duration: {mae_loan_duration}')
print(f'R2 Loan Amount: {r2_loan_amount}')
print(f'R2 Loan Duration: {r2_loan_duration}')


MSE Loan Amount: 174140364.96752644
MSE Loan Duration: 609.2501436
MAE Loan Amount: 9996.168985
MAE Loan Duration: 19.47523
R2 Loan Amount: -0.03332321159522844
R2 Loan Duration: -0.015146207112411325


In [163]:
mse_loan_amount = mean_squared_error(y_test["LoanAmount"], y_pred2[:, 0])
mse_loan_duration = mean_squared_error(y_test["LoanDuration"], y_pred2[:, 1])
mae_loan_amount = mean_absolute_error(y_test["LoanAmount"], y_pred2[:, 0])
mae_loan_duration = mean_absolute_error(y_test["LoanDuration"], y_pred2[:, 1])
r2_loan_amount = r2_score(y_test["LoanAmount"], y_pred2[:, 0])
r2_loan_duration = r2_score(y_test["LoanDuration"], y_pred2[:, 1])

print(f'MSE Loan Amount: {mse_loan_amount}')
print(f'MSE Loan Duration: {mse_loan_duration}')
print(f'MAE Loan Amount: {mae_loan_amount}')
print(f'MAE Loan Duration: {mae_loan_duration}')
print(f'R2 Loan Amount: {r2_loan_amount}')
print(f'R2 Loan Duration: {r2_loan_duration}')


MSE Loan Amount: 169889687.71225494
MSE Loan Duration: 604.2975614553609
MAE Loan Amount: 9732.748928253184
MAE Loan Duration: 19.43154475737809
R2 Loan Amount: -0.008100320431017272
R2 Loan Duration: -0.006894104043817073


In [148]:
y_pred = pd.DataFrame(y_pred)
y_pred.rename(columns={0:'LoanAmount'},inplace=True)
y_pred.sort_values(by=['LoanAmount'], inplace=True)

y_train.rename(columns={0:'LoanAmount'},inplace=True)
y_train.sort_values(by=['LoanAmount'], inplace=True)

In [144]:
clientdata = {'Age': [20],
              'AnnualIncome': [20000],
              'EmploymentStatus': [1],
              'EducationLevel': [2],
              'Experience': [15],
              'MaritalStatus': [1],
              'NumberOfDependents': [2],
              'MonthlyDebtPayments': [300],
              'BankruptcyHistory': [0.1],
              'SavingsAccountBalance': [5000],
              'CheckingAccountBalance': [2000],
              'TotalAssets': [9.696440e+04],
              'TotalLiabilities': [3.625241e+04],
              'MonthlyIncome': [5000],
              'JobTenure': [3],
              'NetWorth': [7.229432e+03],
              'MonthlyLoanPayment': [500]
        }

client = pd.DataFrame(clientdata)

In [146]:
y_val = model.predict(client)
y = pd.DataFrame(y_val)

In [147]:
wynik = y_val[:,0] / y_val[:,1]

print(f"Maksymalna zdolność kredytowa: {y.iloc[0, 0]:.2f}")
print(f"Minimalna ilość miesięcy kredytu(dla kwoty maksymalnej) {np.ceil(y.iloc[0,1]):.0f}")
print(f"Maksymalna rata miesięczna: {wynik[0]:.2f} dolary")

Maksymalna zdolność kredytowa: 16950.90
Minimalna ilość miesięcy kredytu(dla kwoty maksymalnej) 66
Maksymalna rata miesięczna: 259.19 dolary
